# This Notebook Is for Exploring Importing Data from IEX into Apache Cassandra
## Future works will be converted into python scripts which will be more easily 'dockerized' for managing requirements and run environments
### Although Cassandra's advantages are more in enterprise-level write performance, I think its widespread use and compatibility with DFS will make it a better option than Mongo for any crazy timelines where this turns into anything

In [7]:
# Import Requirements
import requests
import csv
from bs4 import BeautifulSoup

C:\Users\micha\Anaconda3\envs\py36\lib\site-packages\requests\__init__.py:91: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [87]:
#Use requests session to pull Wikipedia's S&P500 list page, this will hopefully not change too much or else this might break
session = requests.Session()
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
response = session.get(url)
#Pull the content (html) of the HTTP response object into BSoup and use the html parser
soup = BeautifulSoup(response.content,'html.parser')

In [110]:
#Utilize the constituents id to locate the relavant HTML info
const_table = soup.find(id='constituents')
#More parsing...
table_row = const_table.find_all('tr')[1:] #Use of list indexing removes the table header which is also a tr
#Create an empty list to receive the tickers for the S&P500 companies
sp500 = []
#Loop over BSoup result set, each 'row' being a BSoup object that can be further parsed into table data
for row in table_row:
    columns = row.find_all('td')
    #Based on the layout of the table, the second column will contain the symbols
    symbol_col = columns[1]
    sp500.append(symbol_col.text)
#Confirm the length of the list as 505
print(len(sp500))

505


In [139]:
#Write list to a csv file for durability
with open ('./sp500.csv','w',newline='') as csvfile:
    csvwriter = csv.writer(csvfile,delimiter=',')
    for ticker in sp500:
        csvwriter.writerow([ticker])

In [142]:
#Check the csv file
with open('./sp500.csv','r',newline='') as csvfile:
    csvreader = csv.reader(csvfile)
    counter = 0
    for row in csvreader:
        counter += 1
        print(row, counter)

['MMM'] 1
['ABT'] 2
['ABBV'] 3
['ABMD'] 4
['ACN'] 5
['ATVI'] 6
['ADBE'] 7
['AMD'] 8
['AAP'] 9
['AES'] 10
['AMG'] 11
['AFL'] 12
['A'] 13
['APD'] 14
['AKAM'] 15
['ALK'] 16
['ALB'] 17
['ARE'] 18
['ALXN'] 19
['ALGN'] 20
['ALLE'] 21
['AGN'] 22
['ADS'] 23
['LNT'] 24
['ALL'] 25
['GOOGL'] 26
['GOOG'] 27
['MO'] 28
['AMZN'] 29
['AEE'] 30
['AAL'] 31
['AEP'] 32
['AXP'] 33
['AIG'] 34
['AMT'] 35
['AWK'] 36
['AMP'] 37
['ABC'] 38
['AME'] 39
['AMGN'] 40
['APH'] 41
['APC'] 42
['ADI'] 43
['ANSS'] 44
['ANTM'] 45
['AON'] 46
['AOS'] 47
['APA'] 48
['AIV'] 49
['AAPL'] 50
['AMAT'] 51
['APTV'] 52
['ADM'] 53
['ARNC'] 54
['ANET'] 55
['AJG'] 56
['AIZ'] 57
['T'] 58
['ADSK'] 59
['ADP'] 60
['AZO'] 61
['AVB'] 62
['AVY'] 63
['BHGE'] 64
['BLL'] 65
['BAC'] 66
['BK'] 67
['BAX'] 68
['BBT'] 69
['BDX'] 70
['BRK-B'] 71
['BBY'] 72
['BIIB'] 73
['BLK'] 74
['HRB'] 75
['BA'] 76
['BKNG'] 77
['BWA'] 78
['BXP'] 79
['BSX'] 80
['BHF'] 81
['BMY'] 82
['AVGO'] 83
['BR'] 84
['BF-B'] 85
['CHRW'] 86
['COG'] 87
['CDNS'] 88
['CPB'] 89
['COF'] 

In [5]:
#Import Cassandra driver
from cassandra.cluster import Cluster
from cassandra.cqlengine.management import sync_table

In [4]:
#Establish a Cluster object that will connect to the cassandra instances
cluster = Cluster()
session = cluster.connect('sp500')

In [ ]:
#Import the ORM Model for future mapping, but I'm going to try and make this work iteratively
from cassandra.cqlengine.models import Model

In [9]:
#Open the S&P500 csv and begin iterating over ticker symbols
list_of_sp500 = [] #This doesn't need to be done twice, but I'm keeping the cells modular for easier copypasta later
with open('./sp500.csv','r',newline='') as csvfile:
    csvreader = csv.reader(csvfile)
    for row in csvreader:
        list_of_sp500.append(row[0]) #The reader will return an interative object so we reference the sole value

['MMM', 'ABT', 'ABBV', 'ABMD', 'ACN', 'ATVI', 'ADBE', 'AMD', 'AAP', 'AES', 'AMG', 'AFL', 'A', 'APD', 'AKAM', 'ALK', 'ALB', 'ARE', 'ALXN', 'ALGN', 'ALLE', 'AGN', 'ADS', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AEE', 'AAL', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'ABC', 'AME', 'AMGN', 'APH', 'APC', 'ADI', 'ANSS', 'ANTM', 'AON', 'AOS', 'APA', 'AIV', 'AAPL', 'AMAT', 'APTV', 'ADM', 'ARNC', 'ANET', 'AJG', 'AIZ', 'T', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY', 'BHGE', 'BLL', 'BAC', 'BK', 'BAX', 'BBT', 'BDX', 'BRK-B', 'BBY', 'BIIB', 'BLK', 'HRB', 'BA', 'BKNG', 'BWA', 'BXP', 'BSX', 'BHF', 'BMY', 'AVGO', 'BR', 'BF-B', 'CHRW', 'COG', 'CDNS', 'CPB', 'COF', 'CPRI', 'CAH', 'KMX', 'CCL', 'CAT', 'CBOE', 'CBRE', 'CBS', 'CE', 'CELG', 'CNC', 'CNP', 'CTL', 'CERN', 'CF', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'XEC', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CTXS', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CMA', 'CAG', 'CXO', 'COP', 'ED', 'STZ', 'COO', 'CPRT', 'GLW', 'COST', 'COTY', 'CCI', 'CS

In [11]:
#Define a function that will accept a ticker string and request
# the 5 year JSON history from IEX
def GetAndReturnJSON(ticker):
    '''This function accepts a string corresponding to a stock ticker'''
    #Build out the url string from the ticker for 5 year history
    url = "https://api.iextrading.com/1.0/stock/"+ticker+"/chart/5y"
    session = requests.Session()
    response = session.get(url)
    return response.json()

#TODO -- Build the function that will pass the JSON into a Cassandra Model
# and then create that row in the S&P500 table

[{'change': 1.1176,
  'changeOverTime': 0,
  'changePercent': 1.588,
  'close': 71.501,
  'date': '2014-02-13',
  'high': 71.5562,
  'label': 'Feb 13, 14',
  'low': 70.1575,
  'open': 70.2179,
  'unadjustedVolume': 10994308,
  'volume': 76960156,
  'vwap': 71.1055},
 {'change': -0.057788,
  'changeOverTime': -0.0008083803023733974,
  'changePercent': -0.081,
  'close': 71.4432,
  'date': '2014-02-14',
  'high': 71.7046,
  'label': 'Feb 14, 14',
  'low': 71.0781,
  'open': 71.2436,
  'unadjustedVolume': 9781148,
  'volume': 68468036,
  'vwap': 71.378},
 {'change': 0.262666,
  'changeOverTime': 0.002865694186095229,
  'changePercent': 0.368,
  'close': 71.7059,
  'date': '2014-02-18',
  'high': 72.3888,
  'label': 'Feb 18, 14',
  'low': 71.656,
  'open': 71.7072,
  'unadjustedVolume': 9329464,
  'volume': 65306248,
  'vwap': 71.9914},
 {'change': -1.1321,
  'changeOverTime': -0.01296765080208667,
  'changePercent': -1.579,
  'close': 70.5738,
  'date': '2014-02-19',
  'high': 71.8241,
  